MIT-BIH Arrhythmia Database의 213번 환자를 예시로 V peak을 추출하는 데이터 처리 과정 예시입니다.

In [1]:
import os
import pandas as pd
import numpy as np
import csv
import matplotlib.pylab as plt
from scipy.signal import resample

In [2]:
def downsample(data, oldFS, newFS):
    newNumSamples = int((data.shape[0] / oldFS) * newFS)
    newData = pd.DataFrame(resample(data, newNumSamples))
    return newData

In [3]:
data = pd.read_csv("./annotation_arrhythmia/213.csv")

In [4]:
data.head()

,Unnamed: 0,Time,Sample,#,Type,Sub,Chan,Num,Aux
0,NaN,0:00.050,18,+,0,0,0,(N,NaN
1,NaN,0:00.264,95,N,0,0,0,NaN,NaN
2,NaN,0:00.703,253,N,0,0,0,NaN,NaN
3,NaN,0:01.328,478,N,0,0,0,NaN,NaN
4,NaN,0:01.869,673,N,0,0,0,NaN,NaN


In [5]:
data['#'].value_counts()

N    2641
F     362
V     220
+      43
A      25
a       3
Name: #, dtype: int64

In [6]:
shape_row=data['#'].values
V_index=np.where(shape_row=='V')
ann_index=data['Sample'].values
V_ann_index = ann_index[V_index]
print(V_ann_index)

[ 43751  49998  53940  54341  54720  57445  60165  60594  63318  67635
  68030  71178  71534  75896  77867  87301  90032  93152  95929  96314
  96688  99823 102185 106131 106527 108512 108908 111630 112028 114796
 115191 115590 117520 117945 120663 159674 162438 162827 165201 165595
 167169 167547 167961 169143 169946 170712 171123 171520 173495 173896
 174268 176651 177029 177450 179820 180193 182189 182568 184918 186519
 188090 188475 192029 192807 194777 197911 200689 203836 204238 206003
 206829 219471 219878 220256 222638 224994 225384 225777 228154 228581
 230956 275705 276105 278108 278497 278903 279271 282458 282886 283285
 285669 286069 286436 286868 287261 289627 291587 292418 292807 295181
 295565 297969 298379 300766 305961 308376 308761 309166 311583 311985
 315140 316376 318438 322328 322505 324700 327105 329516 332308 376292
 376690 377087 377477 379318 379489 379676 379901 381497 381930 382313
 383923 384697 385932 386325 386736 387129 387528 387935 388734 389008
 39013

In [8]:
raw = pd.read_csv('./ECG_rawfile_arrhythmia/213raw.csv',header=None,names= ['time','ecg_ch1','ecg_ch2'])
ch1=raw['ecg_ch1']
ch1_val = ch1.values
ch1_list = ch1_val.tolist()
ch1_array = np.asarray(ch1_list)

In [9]:
v = np.zeros((0,216),int)
for i in range(0,220):
    q=V_ann_index[i]
    w=int(q)
    e = ch1_array[(w-108):(w+108)]
    v = np.vstack((v,e))
print(v)

[[ 0.295  0.325  0.34  ... -0.27  -0.275 -0.29 ]
 [-0.47  -0.45  -0.41  ... -1.16  -1.09  -1.035]
 [ 0.24   0.24   0.235 ... -0.425 -0.445 -0.45 ]
 ...
 [ 0.28   0.315  0.355 ... -0.125 -0.135 -0.145]
 [ 0.465  0.445  0.435 ... -0.285 -0.295 -0.3  ]
 [-0.525 -0.51  -0.48  ... -1.475 -1.42  -1.37 ]]


In [10]:
newv=v.transpose()
ds=downsample(newv,360,128)
ds_new=ds.transpose()
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
ds_new[:]=scaler.fit_transform(ds_new[:])
ds_new

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.639281,0.672223,0.672831,0.641531,0.640451,0.607754,0.579896,0.566473,0.540488,0.497784,...,0.373449,0.438460,0.491100,0.527876,0.564834,0.598923,0.635117,0.623272,0.644762,0.616102
1,0.177697,0.370376,0.355909,0.430828,0.484237,0.527209,0.576138,0.630944,0.669666,0.683906,...,0.093611,0.107900,0.103391,0.101150,0.117228,0.129052,0.146552,0.151658,0.136942,0.178890
2,0.579198,0.629350,0.641662,0.617150,0.586962,0.578286,0.547737,0.514977,0.496578,0.454440,...,0.381653,0.434003,0.489663,0.519692,0.570148,0.581138,0.597444,0.590715,0.602394,0.533761
3,0.609801,0.664148,0.645747,0.616777,0.586217,0.562994,0.509879,0.477325,0.459020,0.413163,...,0.413239,0.466550,0.502313,0.521236,0.564082,0.562279,0.588762,0.564482,0.550517,0.481183
4,0.517181,0.611700,0.644483,0.673496,0.682479,0.682992,0.699538,0.686285,0.661395,0.635846,...,0.285355,0.340572,0.395170,0.449106,0.498786,0.519941,0.546965,0.528685,0.544235,0.496821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,0.804147,0.865230,0.881348,0.817639,0.763053,0.740254,0.709029,0.679533,0.637844,0.572043,...,0.421618,0.462137,0.505531,0.533334,0.587149,0.591003,0.608017,0.590429,0.619158,0.513599
216,0.435674,0.457663,0.410339,0.361477,0.311578,0.295935,0.251786,0.219481,0.196849,0.171561,...,0.347514,0.413673,0.459057,0.511296,0.573109,0.579035,0.603321,0.599341,0.591518,0.566444
217,0.662986,0.676332,0.743269,0.741472,0.784077,0.769688,0.778717,0.790215,0.780688,0.723117,...,0.351394,0.398647,0.444749,0.489533,0.556486,0.612655,0.663525,0.667045,0.703451,0.693073
218,0.701387,0.701079,0.671483,0.616218,0.603079,0.566368,0.506753,0.498068,0.462832,0.399688,...,0.391031,0.444909,0.478290,0.517846,0.586179,0.605406,0.633634,0.622350,0.644101,0.598592


np.savetxt('Vpeak.csv',ds_new,delimiter=",")
를 통해 csv파일로 추출하여 저장했습니다.